In [1]:
import geopandas as gpd
import pandas as pd 
import json
from area import area

from geopandas.geoseries import *

# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=wfs_forst_verwalt2014@senstadt&type=WFS
forests = gpd.read_file('source/waldflaechen.geojson')
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

isBerlinForest = forests['land'] == 'Berlin'
berlinForests = forests[isBerlinForest]

print(len(forests)) # 11522
print(len(berlinForests)) # 6258
print(berlinForests['area_text'].sum()) # 282036587.22155315
print(forests['area_text'].sum()) # 433585749.9072622 => 43358.5749907 hectar Berlin and Brandenburg
print(forests[(forests.land == 'Berlin')]['area_text'].sum()) # 282036587.22155315 => 28203.658722155315 hectar in Berlin
print(forests[(forests.land == 'Brandenburg')]['area_text'].sum()) # 151549162.68570915

berlinForest = berlinForests.unary_union
berlinForest.crs = "EPSG:5678"
berlinForestJSON = gpd.GeoSeries([berlinForest]).to_json()

berlinForest = json.loads(berlinForestJSON)
berlinForest = berlinForest['features'][0]['geometry']
waldflaeche = area(berlinForest) 
print(waldflaeche) # 162857690.6703506 in m^2

11522
6258
282036587.22155315
433585749.9072622
282036587.22155315
151549162.68570915


KeyboardInterrupt: 

In [ ]:
# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_gruenanlagenbestand@senstadt&type=WFS
open_spaces = gpd.read_file('source/gruenflaechen.geojson')

gruenflaeche = open_spaces['KATASTERFL'].sum()
print(gruenflaeche) # 61475596.6
print(len(open_spaces)) # 2525
gesamt = waldflaeche + gruenflaeche
print(gesamt/10000) # 22433.32872703506

In [ ]:
# source: Geoportal Berlin/Grünanlagenbestand Berlin (einschl. der öffentlichen Spielplätze)
# https://fbinter.stadt-berlin.de/fb/index.jsp?loginkey=alphaDataStart&alphaDataId=s_gruenanlagenbestand@senstadt
data = pd.read_csv("source/gruenflaechen.csv", sep=";").replace('-', '0', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"])
print(len(data)) # 2525

data = data.loc[data['Größe in m² (Kataster)'] > 0.0]
print(data["Größe in m² (Kataster)"].sum() / 10000) # 6147.55966

print(len(data)) # 2518

In [ ]:
# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_gruenanlagenbestand@senstadt&type=WFS
data = gpd.read_file('source/gruenflaechen.geojson')
print(len(data)) # 2525
data = data.loc[data['KATASTERFL'] > 0]
print(data.count())

# # KENNZEICH     2518
# # BEZIRKNAME    2518
# # ORTSTLNAME    2514
# # OBJARTNAME    2518
# # NAMENR        2518
# # NAMEZUSATZ    1368
# # BAUJAHR        292
# # SANIERJAHR      98
# # KATASTERFL    2518
# # WIDMUNG       2400
# # PLANNR        2517
# # PLANNAME      2516
# # ANRECH_GF     2507
# # geometry      2518

print(data['KATASTERFL'].sum()) # 61475596.6